## Morphological Calls(63x) by Post Patch Outcomes

##### Date: 4/26/2018 Updated 63x.csv & shiny.csv

In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

import post_patch as pp #post patch script
#post_patch_column = 'anno.postPatch_label'
#post_patch_pipette_column = 'anno.endPipetteR_label'

#sns.set_style('whitegrid') #Makes a white backround with gray horizontal gridmarks at y-values
#sns.set_context('notebook')

In [2]:
#Reading the CSVs
morph_df = pd.read_csv("C:/users/kumar/allen-institute/personal-projects/csv/63x.csv")
shiny_df = pd.read_csv("C:/users/kumar/allen-institute/personal-projects/csv/shiny_mouse.csv")

In [4]:
#Creating shiny_df
shiny_df = shiny_df.loc[:,'sample_id',
                        'anno.postPatch_label',
                        'anno.endPipetteR_label']

#Creating morph_df
morph_df = morph_df[:,'patched_cell_container',
                    'ims63x_go_no_g',
                    'project']

IndexingError: Too many indexers

In [ ]:
#Inner merge = contains only matching data of both morph_df and shiny_df
final_df = pd.merge(left = morph_df, 
                    right = shiny_df, 
                    left_on = 'patched_cell_container', 
                    right_on = 'anno.sample_id', 
                    how = 'inner')

In [ ]:
final_df.head()

In [ ]:
#Getting date from patched cell container
final_df['Date'] = final_df['patched_cell_container'].str[5:11]

#Changing to string to add 20 in front of strings within date column
final_df['Date'] = final_df['Date'].astype('str')
final_df['Date'] = '20' + final_df['Date']
final_df['Date'] = pd.to_datetime(final_df['Date']) #Converting date to YYYY-MM-DD

final_df = final_df.set_index('Date') #Set date column as index column
final_df.sort_index(inplace = True) #Sort the date column just in case

#start_date = input('Enter the start date for your dataframe (YYYY-MM-DD): ')
#Production Date Start = '2017-10-01' 
start_date = '2017-10-01'
final_df = final_df[start_date:] #Choosing start & end date    

final_df.rename(columns = {'ims63x_go_no_go' : 'morph'}, inplace = True)
final_df = pp.postpatch_reclass(final_df)

In [ ]:
final_df.head()

In [ ]:
final_df.tail()

##### Dropping columns 

In [ ]:
final_df.drop(['patched_cell_container', 'project', 'anno.sample_id', 'anno.endPipetteR_label'], axis=1, inplace=True)

##### Filters columns that have NaN

In [ ]:
final_df.dropna(subset = ['morph'], inplace = True)

In [ ]:
#Categorical
final_df['morph'] = final_df['morph'].astype('category')
final_df['post_patch'] = final_df['post_patch'].astype('category')

#Numerical
#final_df['anno.endPipetteR_label'] = final_df['anno.endPipetteR_label'].astype('float')

##### Date Range for final_df

In [ ]:
final_df.sort_index(inplace = True)

In [ ]:
final_df.head()

In [ ]:
final_df.tail()

##### Info

In [ ]:
final_df.info()

### Creating pivot table 

In [ ]:
final_df1 = final_df[['post_patch', 'morph', 'anno.postPatch_label']]
final_df1.head()

In [ ]:
final_df1 = final_df.groupby(['post_patch', 'morph']).count()

In [ ]:
final_df1

In [ ]:
final_df1 = final_df1.rename(columns = {'anno.postPatch_label' : ''})

In [ ]:
final_df1

In [ ]:
final_df1.reset_index(inplace = True)

In [ ]:
final_df1

In [ ]:
final_df1 = final_df1.pivot_table(values=[''], 
                                  index=['post_patch'], 
                                  columns = ['morph'], 
                                  aggfunc='sum')

In [ ]:
final_df1

##### Adding total counts 

In [ ]:
#Rusty's Method figure it out later
post_patch_totals = list(final_df1.sum(1))
post_patch_totals = [int(x) for x in post_patch_totals]

##### Aaron's method for Plot: Morphological Calls(63x) by Post Patch Outcomes

In [ ]:
x = ['Nuc-high seal', 'Nuc-low seal', 'No-high seal', 'No-low seal', 'Entire cell']
morph = ['63x go', '63x no go']
#color = ["r", "g"]

bottom = np.zeros(len(x))
y = np.zeros(len(x))
for mp in morph:
    bottom = bottom + y
    y = [len(final_df[(final_df['post_patch'] == f) & (final_df['morph'] == mp)]) for f in x]
    plt.bar(x, y, bottom = bottom, label = mp)
    
plt.xlabel('Post Patch Outcomes')
plt.ylabel('Count of Morphological Calls')
plt.legend()
plt.title('Count of Morphological Calls by Post Patch Outcomes')
#plt.savefig("count_patch_duration_vs_morph")

##### Same Plot as Aaron's Method

In [ ]:
my_colors = ['#74c476', '#de2d26']
bp = final_df1.plot.bar(stacked=True, color = my_colors)

bp.legend(loc = 0, bbox_to_anchor=(1, 1))
bp.set(xlabel = 'Post Patch Outcomes', ylabel = 'Count')

##### Vertical Bar Plot

##### Changed final_df1 from ratios to percentages 

In [ ]:
final_df1 = final_df1.div(final_df1.sum(1), axis=0) * 100

In [ ]:
final_df1

### Stacked Percentage Bar Plot: Morphological Calls(63x) by Post Patch Outcomes

### Vertical Bar Plot with Percentages

In [ ]:
#order = ['Nuc-high seal', 'Nuc-low seal', 'No-high seal', 'No-low seal', 'Entire cell']
#.loc[order] lets you choose the order

my_colors = ['#74c476', '#de2d26']
bp = final_df1.plot.bar(stacked = True, 
                        color = my_colors, 
                        figsize= (10,8),
                        rot = 0)

bp.legend(['63x go', '63x no go'],
          loc = 0, 
          bbox_to_anchor = (1, 1), 
          frameon = True, 
          shadow = True,
          fontsize = 13)

plt.subplots_adjust(left = 0.1, right = 0.8, bottom = None, top = None, wspace=None, hspace=None)
#This helps if axis labels are getting cutoff when saving final image

#Rusty's Method figure it out later
rects = bp.patches
labels = post_patch_totals

#Rusty's Method figure it out later
for rect, label in zip (rects, labels):
    height = 100
    x_value = rect.get_x() + rect.get_width() / 2
    bp.text(rect.get_x() + rect.get_width()/2, height, label, ha='center', va='bottom', size = 12)

bp.set_title('Percentage of Morphological Calls(63x) by Post Patch Outcomes')
bp.set(xlabel = 'Post Patch Outcomes', ylabel = 'Percentage')
plt.savefig('Stacked_Percentage_Barplot_M_by_PP')

##### Horizontal Bar Plot with Percentages

In [ ]:
my_colors = ['#74c476', '#de2d26']
bp = final_df1.plot(kind = 'barh', 
                    stacked = True, 
                    color = my_colors,
                    figsize= (8,5))

bp.legend(['63x go', '63x no go'],
          loc = 0, 
          bbox_to_anchor = (1, 1), 
          frameon = True, 
          shadow = True,
          fontsize = 10)

bp.set(xlabel = 'Percentage', ylabel = 'Post Patch Outcomes')

### Trial plot to order post patch outcomes

In [ ]:
#TRY to order the post patch outcomes for .plot.bar(stacked = True)

In [ ]:
order = ['Nuc-high seal', 'Nuc-low seal', 'No-high seal', 'No-low seal', 'Entire cell']
#.loc[order] lets you choose the order

my_colors = ['#74c476', '#de2d26']
bp = final_df1.loc[order].plot.bar(stacked = True, 
                    color = my_colors,
                    figsize= (8,5),
                    rot = 0)

bp.legend(['63x go', '63x no go'],
          loc = 0, 
          bbox_to_anchor = (1, 1), 
          frameon = True, 
          shadow = True,
          fontsize = 10)

bp.set(xlabel = 'Percentage', ylabel = 'Post Patch Outcomes')